<a href="https://colab.research.google.com/github/Jorgecardetegit/DiseaseClassifier/blob/main/Simple_CNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split

from glob import glob
import random

In [3]:
gen_dir = "/content/drive/MyDrive/Apple disease proyect/dataset"
categories = os.listdir(gen_dir)
categories = [category.split('___')[1].lower() for category in categories]

apple_scab = glob('/content/drive/MyDrive/Apple disease proyect/dataset/Apple___Apple_scab/*.JPG')
black_rot = glob("/content/drive/MyDrive/Apple disease proyect/dataset/Apple___Black_rot/*.JPG")
cedar_apple_rust = glob("/content/drive/MyDrive/Apple disease proyect/dataset/Apple___Cedar_apple_rust/*.JPG")
healthy = glob("/content/drive/MyDrive/Apple disease proyect/dataset/Apple___healthy/*.JPG")

# 1. Preprocessing

### Normalizing or Standardizing and rescaling

All the images in the dataset have the same size so it is not strictly necessary to Standardize or even Normalize the dataset, nevertheless I will normalize the images in order to converge faster during training and improve generalization. I won´t standardize it.

In addition the images have a size of 255*255 which is quite standard and suitable for the models I am going to fit so I won´t rescale the images.

### Data Augmentation
As stated in the EDA anlaysis the main problem in the dataset is the imbalance of data amount presented in each category. In order to deal with this issue I will use the method of data augmentation to artificially incraese the number of images in the categories with less images.

Is important to be carefull with this method, since it can provoke overfitting. To start with I will use the following technique with each variable:

- apple_scab: Apply mild to moderate augmentation. An increase of 20-50% in the number of images will be the starting point.

- black_rot: Will also apply mild to moderate augmentation. An increase of 20-50% in the number of images will be the starting point.

- cedar_apple_rust: Apply moderate to aggressive data augmentation to this class to increase the number of samples. I aim to at least double the number of images in this class

- healthy: No augmentation will be applied for now.

### Splitting ratios
- Train: 70%
- Validation: 15%
- Test: 15%

### Parameters definition

- IMAGE_WIDTH = 255
- IMAGE_HEIGHT = 255
- NUM_CHANNELS = 3
- BATCH_SIZE = 32
- EPOCHS = 50

## Preprocessing Class

In [2]:
class preprocessingClass:
  def __init__(self):
    self.IMAGE_HEIGHT = 255
    self.IMAGE_WIDTH = 255
    self.NUM_CHANNELS = 3
    self.BATCH_SIZE = 32
    self.EPOCHS = 50
    self.CHANNELS = 3

  def preprocessing(self, directory):
    dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory,
    shuffle = True,
    image_size = (self.IMAGE_HEIGHT,self.IMAGE_WIDTH),
    batch_size = self.BATCH_SIZE
    )

    return dataset

  def split_dataset(self, directory, train_split=0.7, val_split=0.15, test_split=0.15, shuffle = True, shuffle_size = 10000):
    dataset = self.preprocessing(directory)
    num_samples = len(dataset)

    if shuffle:
      dataset = dataset.shuffle(shuffle_size, seed = 12)

    train_size = int(train_split * num_samples)
    val_size = int(val_split * num_samples)
    test_size = int(test_split * num_samples)

    train_dataset = dataset.take(train_size)
    val_dataset = dataset.skip(train_size).take(val_size)
    test_dataset = dataset.skip(train_size + val_size).take(test_size)

    return train_dataset, val_dataset, test_dataset

  def train_gen(self,train_ds,rescaling_size):

    train_datagen = ImageDataGenerator(
        rescale = 1./rescaling_size,
        horizontal_flip = True,
        rotation_range = 10
    )

    tain_generator = train_datagen.flow_from_directory(
        "data/train",
        target_size = (self.IMAGE_WIDTH, self.IMAGE_HEIGHT),
        batch_size = self.BATCH_SIZE,
        class_mode = "sparse",
        save_to_dir = "AugmentedImages"
    )

  def optimization(self,train_ds, val_ds, test_ds):
    train_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
    val_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
    test_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)


  def dataAugmentation(self, train_ds):
    data_augmentation_sequential = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2)
    ])

    train_ds = train_ds.map(lambda x, y: (data_augmentation_sequential(x, training=True), y)).prefetch(buffer_size=tf.data.AUTOTUNE)

    return train_ds

  def ResizeRescale(self, train_ds, val_ds, test_ds):
    resize_rescale_sequential = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(self.IMAGE_WIDTH, self.IMAGE_HEIGHT),
    layers.experimental.preprocessing.Rescaling(1.0/255)
    ])

    train_ds = train_ds.map(lambda x, y: (resize_rescale_sequential(x), y))
    val_ds = val_ds.map(lambda x, y: (resize_rescale_sequential(x), y))
    test_ds = test_ds.map(lambda x, y: (resize_rescale_sequential(x), y))

    return train_ds, val_ds, test_ds

In [3]:
preprocessMethod = preprocessingClass()
directory = "/content/drive/MyDrive/Apple disease proyect/dataset"

# 1. Process the dataset -----------------------------------------------------------------------------------------------------------------
preprocessMethod.preprocessing(directory)

# 2. Split the dataset -------------------------------------------------------------------------------------------------------------------
train_ds, val_ds, test_ds = preprocessMethod.split_dataset(directory, train_split = 0.7, val_split = 0.15, test_split = 0.15,
                                                             shuffle = True, shuffle_size = 10000)
# 3. Optimize the dataset ----------------------------------------------------------------------------------------------------------------
preprocessMethod.optimization(train_ds, val_ds, test_ds)

# 4. Data Augmentation --------------------------------------------------------------------------------------------------------------------
train_ds = preprocessMethod.dataAugmentation(train_ds)

# 5. Resizing and Rescaling ---------------------------------------------------------------------------------------------------------------
train_ds, val_ds, test_ds = preprocessMethod.ResizeRescale(train_ds, val_ds, test_ds)


Found 3171 files belonging to 4 classes.
Found 3171 files belonging to 4 classes.


### Model Architecture

In [9]:
IMAGE_HEIGHT = 255
IMAGE_WIDTH = 255
NUM_CHANNELS = 3
BATCH_SIZE = 32
EPOCHS = 50
CHANNELS = 3

input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, CHANNELS)
n_classes = 4

model = tf.keras.Sequential([
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

model.build(input_shape=input_shape)

In [10]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 253, 253, 32)      896       
                                                                 
 max_pooling2d_18 (MaxPooli  (None, 126, 126, 32)      0         
 ng2D)                                                           
                                                                 
 conv2d_19 (Conv2D)          (None, 124, 124, 64)      18496     
                                                                 
 max_pooling2d_19 (MaxPooli  (None, 62, 62, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_20 (Conv2D)          (None, 60, 60, 64)        36928     
                                                                 
 max_pooling2d_20 (MaxPooli  (None, 30, 30, 64)       

In [11]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

## Model training

In [12]:
history = model.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    validation_data=val_ds,
    verbose=1,
    epochs=50,
)

Epoch 1/50
 6/70 [=>............................] - ETA: 4:37 - loss: 1.2750 - accuracy: 0.5000

KeyboardInterrupt: ignored

In [ ]:
scores = model.evaluate(test_ds)

In [ ]:
class DataAugmentation():

  def shuffleBatchCategory(self,category):
      shuffled_dataset = random.sample(category, len(category))

      return shuffled_dataset

  def transformations(self, image_path):
      image = tf.io.read_file(image_path)
      image = tf.image.decode_image(image, channels=3)

      image = tf.image.random_flip_left_right(image)
      image = tf.image.rot90(image, k=tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))
      # Add the transoformations here

      return image

  def augmentation(self,image_list, num_image=None):
      if num_image is None:
          num_image = len(image_list)

      augmented_images = [self.transformations(image) for image in image_list]
      combined_images = image_list + augmented_images

      return combined_images

  def save_images(self, image_list, output_dir):
      os.makedirs(output_dir, exist_ok=True)

      for i, image in enumerate(image_list):
          transformed_filename = f"transformed_image_{i}.jpg"            # Genera un nombre de archivo único con sufijo de transformación y extensión .jpg
          file_path = os.path.join(output_dir, transformed_filename)

          image = tf.cast(image, tf.uint8)                               # Convierte la imagen a formato uint8 (0-255) y la guarda como JPEG
          encoded_image = tf.image.encode_jpeg(image)

          with open(file_path, "wb") as f:                               # Escribe el archivo JPEG en el directorio de salida
              f.write(encoded_image.numpy())

  def concatenate(category1, category2, category3):
      general_dataset = category1 + category2 + category3

      return general_dataset


In [ ]:
DataAugmentor = DataAugmentation()

# Apple_scab_augmentation----------------------------------------------------------------------------------------------------------------------------------
shuffled_dataset_apple_scab = DataAugmentor.shuffleBatchCategory(category = apple_scab)

augmented_dataset_apple_scab = DataAugmentor.augmentation(image_list = shuffled_dataset_apple_scab,num_image = round(len(shuffled_dataset_apple_scab)/2))

DataAugmentor.save_images(image_list = augmented_dataset_apple_scab,
                          output_dir ="/content/drive/MyDrive/Apple disease proyect/dataset/Apple___Apple_scab" )

# Black_rot augmentation----------------------------------------------------------------------------------------------------------------------------------
shuffled_dataset_black_rot = DataAugmentor.shuffleBatchCategory(category = black_rot)

augmented_dataset_black_rot = DataAugmentor.augmentation(image_list = shuffled_dataset_black_rot, num_image = round(len(shuffled_dataset_black_rot)/2))

DataAugmentor.save_images(image_list = augmented_dataset_black_rot,
                         output_dir ="/content/drive/MyDrive/Apple disease proyect/dataset/Apple___Black_rot" )

# Cedar_apple_rust augmentation---------------------------------------------------------------------------------------------------------------------------
shuffled_dataset_cedar_apple_rust = DataAugmentor.shuffleBatchCategory(category = cedar_apple_rust)

augmented_dataset_cedar_apple_rust = DataAugmentor.augmentation(image_list = shuffled_dataset_cedar_apple_rust)

DataAugmentor.save_images(image_list = augmented_dataset_cedar_apple_rust,
                          output_dir ="/content/drive/MyDrive/Apple disease proyect/dataset/Apple___Cedar_apple_rust" )